# Drift - Comparing to Bmad

Simple 1 m drift.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pmd_beamphysics import single_particle
from pmd_beamphysics.units import mec2
from pytao import Tao

import impact.z as IZ
from impact.tests.z.conftest import bmad_files
from impact.z.interfaces.bmad import export_particles as tao_export_particles

In [ ]:
x0 = 0.001
y0 = 0.002
z0 = 0  # By definition
t0 = 0.003
px0 = 1e6
py0 = 2e6
energy0 = 10e6
pz0 = np.sqrt(energy0**2 - px0**2 - py0**2 - mec2**2)

P0 = single_particle(px=px0, py=py0, pz=pz0, x=x0, y=y0, z=z0, t=t0)

In [ ]:
!cat $bmad_files/drift.bmad

In [ ]:
tao = Tao(lattice_file=bmad_files / "drift.bmad", plot="mpl")

In [ ]:
P0.write("drift_initial_particles.h5")
for cmd in [
    "set beam_init position_file = drift_initial_particles.h5",
    f"set beam_init n_particle = {len(P0)}",
    f"set beam_init bunch_charge = {P0.charge}",
    "set beam_init saved_at = beginning d",
    "set global track_type = single",
    "set global track_type = beam",
]:
    res = tao.cmd(cmd)
    if res:
        print(res)

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
I = IZ.ImpactZ.from_tao(
    tao, use_temp_dir=False, workdir="./drift-bmad", initial_particles=P0
)

In [ ]:
output = I.run()

In [ ]:
I.input

In [ ]:
I.output

In [ ]:
P = I.output.particles["final_particles"]
Ptao = tao_export_particles(tao, "END")

In [ ]:
stats = I.output.stats
z = stats.z
x = stats.mean_x
y = stats.mean_y

x_tao = tao.bunch_comb("x")
y_tao = tao.bunch_comb("y")
s_tao = tao.bunch_comb("s")

fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 4))
ax1.plot(z, x, label="Impact-Z")
ax1.plot(s_tao, x_tao, "--", label="Tao")
ax1.set_ylabel(r"$x$ (m)")

ax2.plot(z, y, label="Impact-Z")
ax2.plot(s_tao, y_tao, "--", label="Tao")
ax2.set_ylabel(r"$y$ (m)")

ax2.set_xlabel(r"$s$ (m)")

plt.legend();